In [86]:
#codigo para calcular los tamanos del opam two stage PAG 351
import numpy as np

vdd = 1.8             #voltage input feeding
vss = 0                 #voltage in gnd
vpp = 2.8               #peak to peak voltage

vout1 = 0.5 * vpp          #voltage node j needs to sustain at vout1
vx = 0.1 * vout1          #voltage node x needs to sustain
P = 1e-3       #Power budget 1mW 

mu_n = 0.0318614          #mobility V^-1 pmos (sky130) https://web.open-source-silicon.dev/t/423504/hello-what-are-the-standard-values-for-kn-for-nmos-and-kp-fo#3251cabc-ceaa-427e-8b01-5dcbebea7a24
mu_p = 0.01            #mobility V^-1 pmos (sky130)

eox = 3.9 * 8.854e-12   #Farads/Meter     https://inst.eecs.berkeley.edu/~ee143/fa16/lectures/Lecture13-MOSFET%20Devices.pdf
tox = 4.148e-9          #sky130            https://web.open-source-silicon.dev/t/8120582/hi-everyone-what-is-the-t-ox-for-sky130-fd-pr-nfet-01v8#ebac9993-85a9-41a0-b851-f1f526c60349
Cox = eox/tox             #gate-oxide capacitance per unit area (sky130)
mu_nCox = mu_n * Cox    #mobility for nmos times Oxide capacitance per unit area A/V^2
mu_pCox = mu_p * Cox      #mobility for pmos times Oxide capacitance A/V^2

L = 0.150       #effective length of transistor micrometers
vthn = 0.769           #threshold voltage for nmos [V]
vthp = 0.624           #threshold voltage for pmos absolute value[V]



In [87]:
mu_pCox

8.32463837994214e-05

In [88]:
#Computing voltage overdrives for inner stage

v_branch_inner = vdd - vx
vod3 = (0.55) * v_branch_inner
vod1 = (0.25) * v_branch_inner
vod9 = (0.2) * v_branch_inner

#computing voltage overdrive for outter stage
v_branch_out = vdd - vout1
vod5 = (1/4) * v_branch_out
vod7 = (3/4) * v_branch_out



In [89]:
vod9

0.3320000000000001

In [90]:
I = P/vdd           #available current for the whole device
I_M1_M8 = ( I )/ 8   #assigning 96% for m1=m8 transiistors
I_M9 = I_M1_M8 * 2

In [91]:
I_M9

0.0001388888888888889

In [92]:
#Computing vb1 and vb2
vb1 = vdd + vss - (vod3 + vthp)
vb2 = vdd + vss - (vod5 + vod7 + vthn)

In [93]:
vb2

0.631

In [94]:
#Computing transistor size
def WL(Id, muCox, vod):
    S = (2 * Id) / (muCox * (vod**2))
    return S

S1 = WL(I_M1_M8, mu_nCox, vod1)
S3 = WL(I_M1_M8, mu_pCox, vod3)
S5 = WL(I_M1_M8, mu_pCox, vod5)
S7 = WL(I_M1_M8, mu_nCox, vod7)
S9 = WL(I_M9, mu_nCox, vod9)



In [95]:
S9

9.5014765306591

In [96]:
W3

0.3002284406617604

In [97]:
#Computing size of transistor M10 current mirror
RI = 1000
S10 = (S9 * vdd) / (RI * I_M9)

In [98]:
S10

123.13913583734191

In [99]:
W1 = S1*L
W3 = S3*L
W5 = S5*L
W7 = S7*L
W9 = S9*L
W10 = S10*L

In [111]:
W10

18.470870375601287

In [100]:
#Computing feedback resistors R1, R2, R3, R4

R12 = 2 * vb1/I_M9


In [101]:
R12

3787.1999999999985

In [102]:
#computing gains of each transistor

def gm(Id, vod):
    gm = 2 * (Id/vod)       #PAG 636 razhavi
    return gm

gm1 = gm(I_M1_M8, vod1)
gm3 = gm(I_M1_M8, vod3)
gm5 = gm(I_M1_M8, vod5)
gm7 = gm(I_M1_M8, vod7)

In [103]:
1/gm7

2160.0000000000005

In [104]:
#computing output resistance for each transistor
la_n = 0.1          #[V^-1]
la_p = 0.2          #[V^-1]

def ro(Id, la):
    ro = 1/(la * Id)  #pag32 razhavi
    return ro

ro_1278 = ro(I_M1_M8, la_n)
ro_3456 = ro(I_M1_M8, la_p)

In [105]:
#Computing Gain first stage

Av_1 = gm1 * 1/((1/ro_1278) + (1/ro_3456))
Av_2 = gm5 * 1/((1/ro_1278) + (1/ro_3456))
Av = Av_1 * Av_2

Av

1070.9504685408294